In [ ]:
pip install load_dotenv gurobipy pulp

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Load environment variables from .env file
load_dotenv('gurobi.lic')

# Read values from the environment
access_id = os.getenv("WLSACCESSID")
secret = os.getenv("WLSSECRET")
license_id = int(os.getenv("LICENSEID"))

# Set up the Gurobi environment
env = gp.Env(empty=True)
env.setParam("WLSACCESSID", access_id)
env.setParam("WLSSECRET", secret)
env.setParam("LICENSEID", license_id)
env.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2677057
Academic license 2677057 - for non-commercial use only - registered to a0___@itesm.mx


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=2677057>

In [ ]:
import numpy as np
import pulp
import pandas as pd

In [ ]:
from time import thread_time_ns
# Usar CBC con ratioGap del 10% (como optcr = 0.1)
#solver = pulp.PULP_CBC_CMD(msg=True, options=['gapRel=0.1'],timeLimit=60,threads=8)
env.setParam('MIPGap', 0.30)
solver = pulp.GUROBI(
    msg=True,
    threads=64,
    env=env
)
solver.options = ('IIS', 1)

Set parameter MIPGap to value 0.3


In [ ]:
# --- Load Data from CSVs ---
try:
    distance_times_df = pd.read_csv('distance_times.csv', index_col=0)
    plantas_df = pd.read_csv('Plantas.csv', index_col=0)
    plantas_vivero_df = pd.read_csv('Plantas_vivero.csv')
    plantas_vol_df = pd.read_csv('Plantas_vol.csv')
    # Try loading Plantas_supervivencia.csv with 'Plant Type' as index_col
    plantas_supervivencia_df = pd.read_csv('Plantas_supervivencia.csv', index_col='Plant Type')
    distancia_almacen_poligono_df = pd.read_csv('Ruta_almacen.csv')

except KeyError:
    # If 'Plant Type' is not found, try 'Plant_Type' or another common variation
    print("Column 'Plant Type' not found in Plantas_supervivencia.csv. Trying 'Plant_Type'...")
    plantas_supervivencia_df = pd.read_csv('Plantas_supervivencia.csv', index_col='Plant_Type')
except Exception as e:
    print(f"An error occurred while loading CSV files: {e}")
    print("Please ensure all CSV files are correctly named and formatted.")
    exit() # Exit if essential files can't be loaded

In [ ]:
distancia_almacen_poligono_km = {}
for _, row in distancia_almacen_poligono_df.iterrows():
    poligono = row['Poligono']
    distancia = row['Distancia (m)']
    if poligono == 18:
      distancia_almacen_poligono_km['P' + str(int(poligono))] = 0
    else: distancia_almacen_poligono_km['P' + str(int(poligono))] = distancia/1000

selected_keys = ['P17', 'P18', 'P19']
distancia_almacen_poligono_km = {k: distancia_almacen_poligono_km[k] for k in selected_keys if k in distancia_almacen_poligono_km}
distancia_almacen_poligono_km

{'P17': np.float64(0.0024300000000000003),
 'P18': 0,
 'P19': np.float64(0.00329)}

In [ ]:
max_viajes_por_dia = 150  # se debe verificar este valor
viajes_por_dia = range(1, max_viajes_por_dia + 1)
distance_times_df = pd.read_csv('distance_times.csv', index_col=0)

# --- Constantes ---
EQUIVALENTE_PLANTAS_POR_HA = 0.0069
MIN_HA_POR_DIA_DIA_LABORAL = 5
MIN_HA_POR_DIA_SABADO = 0
HORAS_LABOR_DIA_LABORAL = 6
HORAS_LABOR_SABADO = 3
COSTO_PLANTACION_POR_PLANTA = 20
COSTO_CAMION_VIVERO = 4500
CAPACIDAD_CAMION_VIVERO = 8000

STACK_CAMIONETA=2 # CHECAR ESTO
ALTURA_ALMACEN_CM=40*5 # Altura maxima de las plantas por el stackeamiento
CAPACIDAD_ALMACEN_CM3 = 400*ALTURA_ALMACEN_CM
AREA_INSTALACION_TRATAMIENTO_CM2 = 1000000
AREA_BASE_PLANTA_CM2 = 625 # 25 * 25
VOLUMEN_CAMION_CM3 = 199*127*109

VOLUMEN_PLANTA_CM3 = {}
for _, row in plantas_vol_df.iterrows():
    especie = row['Especie']
    volumen = row['cm^3']
    VOLUMEN_PLANTA_CM3[especie] = volumen


COSTO_DIA_ACTIVO = 10000      # ← ajústalo a tu gusto


PROMEDIO_TRATADO_PLANTAS= 40 # ajustar para cada planta despues

TIEMPO_ANTICIPACION_PEDIDO_DIAS = 0

CAPACIDAD_ALMACEN_M2 = 400
DIMENSION_BASE_PLANTA_CM = 25
AREA_INSTALACION_TRATAMIENTO_M2 = 100
DIM_CAMION_CM = (199, 127)
TIEMPO_CARGA_DESCARGA_CAMION_MIN = 30
TIEMPO_TRATAMIENTO_20_MIN = 20
TIEMPO_TRATAMIENTO_60_MIN = 60
DIA_INICIO_PLANTACION_DESDE_LLEGADA = 3
DIA_FIN_PLANTACION_DESDE_LLEGADA = 7
RENDIMIENTO_LITRO_CAMIONETA = 10  # ejemplo km por litro
PRECIO_GASOLINA = 22  # ejemplo pesos por litro

# requerimientos de plantas por polígono
requerimiento_plantas = plantas_df.copy()
requerimiento_plantas.drop('Hectárea', axis=1, inplace=True)
requerimiento_plantas.drop('#_plantas', axis=1, inplace=True)
requerimiento_plantas.index = ['P' + str(i) for i in requerimiento_plantas.index]

requerimientos_plantas_poligono = requerimiento_plantas.T.to_dict('index')

requerimientos_plantas_poligono = {
    e_key: {p_key: e_val[p_key] for p_key in ['P17', 'P18', 'P19'] if p_key in e_val}
    for e_key, e_val in requerimientos_plantas_poligono.items()
}

"""MIN_PLANTAS_POR_TIPO = {
    'E1': 1178,
    'E2': 8508,
    'E3': 1223,
    'E4': 1217,
    'E5': 2427,
    'E6': 2430,
    'E7': 2426,
    'E8': 2435,
    'E9': 3643,
    'E10': 1202
}"""

MIN_PLANTAS_POR_TIPO = {
    especie: sum(poligonos.values())
    for especie, poligonos in requerimientos_plantas_poligono.items()
}

MIN_PLANTAS_POR_POLIGONO = {
    'P17': 853,
    'P18': 1031,
    'P19': 645,
}

M_BIG = sum(MIN_PLANTAS_POR_TIPO.values())   # o cualquier cota > plantas máximas/día


# Constantes derivadas
AREA_PLANTA_M2 = (DIMENSION_BASE_PLANTA_CM / 100) ** 2
PLANTAS_POR_M2_ALMACEN = CAPACIDAD_ALMACEN_M2 / AREA_PLANTA_M2
PLANTAS_POR_M2_TRATAMIENTO = AREA_INSTALACION_TRATAMIENTO_M2 / AREA_PLANTA_M2

# CAPACIDAD_CAMION_ENTREGA_PLANTAS = int(DIM_CAMION_CM[0] / DIMENSION_BASE_PLANTA_CM) * \
#                                  int(DIM_CAMION_CM[1] / DIMENSION_BASE_PLANTA_CM)

CAPACIDAD_CAMION_ENTREGA_PLANTAS = 80

PLANTAS_POR_HORA_TRABAJO = 15

# Horizonte temporal para la planificación
cant_dias = 100

dias = range(1, cant_dias + 1)

DIAS_SEMANA = {
    0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday',
    5: 'Saturday', 6: 'Sunday'
}
DIAS_LABORABLES = [d for d, nombre in DIAS_SEMANA.items() if nombre != 'Sunday']


# --- Procesamiento de datos cargados para entradas del modelo ---

# CORRECCIÓN: Cambiar índices y columnas de distance_times_df para tener prefijo 'P'
distance_times_df.index = ['P' + str(i) for i in distance_times_df.index]
distance_times_df.columns = ['P' + str(c) for c in distance_times_df.columns]

# polígonos y polígono de almacenamiento
## poligonos = distance_times_df.index.tolist()
poligonos = ['P17', 'P18', 'P19']
POLIGONO_ALMACENAMIENTO = 'P18'

# viveros
viveros = ['V1', 'V2', 'V3', 'V4']

# especies (tipos de planta), excluyendo columnas no relacionadas
especies = [col for col in plantas_df.columns if col not in ['Hectárea', '#_plantas']]

# matriz de distancias (copiada del DataFrame modificado)

matriz_distancias = distance_times_df.copy()

# costos de viveros
COSTO_ALTO = 1000  # costo alto artificial para desalentar selección

costos_viveros = {vivero: {} for vivero in viveros}

for index, row in plantas_vivero_df.iterrows():
    especie = row['Especie']
    for vivero in viveros:
        costo = row[vivero]
        costos_viveros[vivero][especie] = costo if costo > 0 else COSTO_ALTO

# tiempos de tratamiento por planta según columna 'Nopal'
tiempo_tratamiento_planta = {}
for _, row in plantas_vol_df.iterrows():
    especie = row['Especie']
    valor_nopal = row['Nopal']
    if valor_nopal == 0:
        tiempo_tratamiento_planta[especie] = TIEMPO_TRATAMIENTO_20_MIN
    elif valor_nopal == 1:
        tiempo_tratamiento_planta[especie] = TIEMPO_TRATAMIENTO_60_MIN
    else:
        print(f"Advertencia: Valor inesperado 'Nopal' para {especie}: {valor_nopal}. Usando 20 minutos por defecto.")
        tiempo_tratamiento_planta[especie] = TIEMPO_TRATAMIENTO_20_MIN

In [ ]:
matriz_distancias = matriz_distancias.loc[["P17", "P18", "P19"], ["P17", "P18", "P19"]]

for i in matriz_distancias.index:
    if i in matriz_distancias.columns:
        matriz_distancias.at[i, i] = 0

In [ ]:
matriz_distancias

,P17,P18,P19
P17,0.000000,1.065408,0.924312
P18,1.065408,0.000000,1.989720
P19,0.924312,1.989720,0.000000


In [ ]:
MIN_PLANTAS_POR_TIPO

{'E1': 115,
 'E2': 805,
 'E3': 116,
 'E4': 115,
 'E5': 231,
 'E6': 230,
 'E7': 229,
 'E8': 230,
 'E9': 344,
 'E10': 114}

In [ ]:
# --- Optimization Model Structure (PuLP) ---
# Create the problem
prob = pulp.LpProblem("Reforestation_Optimization", pulp.LpMinimize)

# --- Decision Variables ---

#   1. Nùmero de plantas e del vivero v que se pidieron el dia d

x = pulp.LpVariable.dicts(
    "NumPlantas",
    ((e, v, d) for e in especies for v in viveros for d in dias),
    lowBound=0,
    cat='Integer'
)

#   2. Número de plantas e en el almacen en el dia d

a = pulp.LpVariable.dicts(
    "PlantasAlmacen",
    ((e, d) for e in especies for d in dias),
    lowBound=0,
    cat='Integer'
)

#   3. Numero de plantas e que se tratan en dia d

t = pulp.LpVariable.dicts(
    "PlantasTratadas",
    ((e, d) for e in especies for d in dias),
    lowBound=0,
    cat='Integer'
)

#   4. Numero de plantas e que se sembraron en el poligono p en el dia d que llegaron el dia l

pl = pulp.LpVariable.dicts(
    "PlantasSembradas",
    ((e, p, d, l) for e in especies for p in poligonos for d in dias for l in dias if 3 <= (d - l) <= 7),
    lowBound=0,
    cat='Integer'
)

#   5. Numero de viajes al poligono p en el dia d

num_v = pulp.LpVariable.dicts(
    "ViajesPoligono",
    ((p, d) for p in poligonos for d in dias),
    lowBound=0,
    cat='Integer'
)

#   6. Variable binaria que establece si ese dia se va al poligono p

vi = pulp.LpVariable.dicts(
    "VisitaPoligono",
    ((p, d) for p in poligonos for d in dias),
    cat='Binary'
)

#   7. Numero de plantas e que se llevaron al poligono p en el dia d en el viaje v

c = pulp.LpVariable.dicts(
    "PlantasTransportadasViaje",
    ((e, p, d, viaje) for e in especies for p in poligonos for d in dias for viaje in viajes_por_dia),
    lowBound=0,
    cat='Integer'
)

#   8. Numero de veces que se pidio en el dia d al vivero v
pi = pulp.LpVariable.dicts(
    "PedidosVivero",
    ((v, d) for v in viveros for d in dias),
    lowBound=0,
    cat='Integer'
)

di = pulp.LpVariable.dicts("DiaActivo", dias, cat='Binary')


pedido_activo = pulp.LpVariable.dicts(
    "PedidoActivo",
    ((v, d) for v in viveros for d in dias),
    cat='Binary'
)


In [ ]:
# --- Objective Function ---

prob += (
    # Costo por pedir plantas al vivero (pi * costo fijo por pedido)
    pulp.lpSum(pi[(v, d)] * COSTO_CAMION_VIVERO for v in viveros for d in dias)
    +
    # Costo por plantas pedidas (sumatoria x * costo planta)
    pulp.lpSum(x[(e, v, d)] * costos_viveros[v][e] for e in especies for v in viveros for d in dias)
    +
    # Costo por plantar plantas (cuádruple sumatoria pl * costo de plantar)
    pulp.lpSum(pl[(e, p, d, l)] * COSTO_PLANTACION_POR_PLANTA for e in especies for p in poligonos for d in dias for l in dias if (e, p, d, l) in pl)
    +
    # Costo gasolina: (distancia / rendimiento) * precio * número de viajes v(p,d)
    pulp.lpSum(
        (distancia_almacen_poligono_km[p] / RENDIMIENTO_LITRO_CAMIONETA) * PRECIO_GASOLINA * num_v[(p, d)]
        for p in poligonos for d in dias
    )
    + COSTO_DIA_ACTIVO * pulp.lpSum(di[d] for d in dias)
), "CostoTotal"

In [ ]:
requerimientos_plantas_poligono

{'E1': {'P17': 39, 'P18': 47, 'P19': 29},
 'E2': {'P17': 272, 'P18': 328, 'P19': 205},
 'E3': {'P17': 39, 'P18': 47, 'P19': 30},
 'E4': {'P17': 39, 'P18': 47, 'P19': 29},
 'E5': {'P17': 78, 'P18': 94, 'P19': 59},
 'E6': {'P17': 78, 'P18': 94, 'P19': 58},
 'E7': {'P17': 77, 'P18': 93, 'P19': 59},
 'E8': {'P17': 77, 'P18': 94, 'P19': 59},
 'E9': {'P17': 116, 'P18': 140, 'P19': 88},
 'E10': {'P17': 38, 'P18': 47, 'P19': 29}}

In [ ]:
# Restricciones

# 1. Pide al menos lo minimo para sembrar las hectareas. especies
for e in especies:
    prob += pulp.lpSum(pl[(e, p, d, l)]
                       for p in poligonos
                       for d in dias
                       for l in dias
                       if (e, p, d, l) in pl and 3 <= (d - l) <= 7) >= MIN_PLANTAS_POR_TIPO[e], f"Minimo_Plantas_{e}"


# 2. Se planta lo minimo por especie en cada poligono

for e in especies:
    for p in poligonos:
        prob += pulp.lpSum(
            pl[(e, p, d, l)]
            for d in dias
            for l in dias
            if (e, p, d, l) in pl and 3 <= (d - l) <= 7
        ) >= requerimientos_plantas_poligono.get(e, {}).get(p, 0), f"Minimo_requerimiento_Plantas_{e}_{p}"


# 2. Lo que plantas no debe ser mayor a lo que tratas

for e in especies:
    for d in dias:
        prob += t[(e, d)] >= pulp.lpSum(
            pl[(e, p, d, l)]
            for p in poligonos
            for l in dias
            if (e, p, d, l) in pl and 3 <= (d - l) <= 7
        ), f"Tratamiento_Suficiente_{e}_{d}"

#3. No puedes tratar mas de lo que tienes en almacen

for e in especies:
    for d in dias:
        prob += a[e, d] >= t[e, d], f"Tratado_no_excede_almacen_{e}_{d}"

#4. Flujo del almacen

for e in especies:
    for d in dias:
        if d == 1:
            prob += a[e, d] == 0, f"Inicial_almacen_{e}" # Dia cero del almacen , se inicializa en 0
        else:
            entradas = pulp.lpSum(
                x[e, v, d - TIEMPO_ANTICIPACION_PEDIDO_DIAS]
                for v in viveros
                if (e, v, d - TIEMPO_ANTICIPACION_PEDIDO_DIAS) in x
            )
            salidas = pulp.lpSum(
                pl[e, p, d, l]
                for p in poligonos
                for l in dias
                if (e, p, d, l) in pl
            )
            prob += a[e, d] == a[e, d - 1] + entradas - salidas, f"Flujo_almacen_{e}_{d}"

#5. Volumen

# Usar esta cuando se tenga un volumen diferente por cada tipo de planta
#for d in dias:
#    prob += (
#        pulp.lpSum(a[e, d] * AREA_PLANTA_M2[e] for e in especies) <= PLANTAS_POR_M2_ALMACEN,
#        f"Restriccion_Volumen_Almacen_dia_{d}"
#    )

for d in dias:
    prob += (
        pulp.lpSum(a[e, d] * VOLUMEN_PLANTA_CM3[e] for e in especies) <= CAPACIDAD_ALMACEN_CM3*100,
        f"Restriccion_Volumen_Almacen_dia_{d}"
    )

for d in dias:
    for p in poligonos:
        for v in viajes_por_dia:
            prob += (pulp.lpSum(c[e,p,d,v] * VOLUMEN_PLANTA_CM3[e]) <=  VOLUMEN_CAMION_CM3*3   ,f"Restriccion_Volumen_Camioneta_{p}_{d}_{v}")

for d in dias:
    prob += (
        pulp.lpSum(t[e, d] * AREA_BASE_PLANTA_CM2) <= AREA_INSTALACION_TRATAMIENTO_CM2*3 ,f"Restriccion_Volumen_Tratado_{e}_{d}_")


# No pueden aparecer plantas mágicas

for e in especies:
    total_plantadas = pulp.lpSum(pl[e, p, d, l] for p in poligonos for d in dias for l in dias if 3 <= (d - l) <= 7)
    total_pedidas = pulp.lpSum(x[e, v, d] for v in viveros for d in dias)

    prob += (
        total_plantadas <= total_pedidas,
        f"Conservacion_Plantas_{e}"
    )

# Tiempos tratados

for d in dias:
    # Determinar si el día es sábado (5) o un día entre semana (0–4)
    dia_semana = (d - 1) % 7  # Asumiendo día 1 es lunes
    horas_laborales = HORAS_LABOR_SABADO if dia_semana == 5 else HORAS_LABOR_DIA_LABORAL
    prob += (
            t[e, d] * PROMEDIO_TRATADO_PLANTAS <= horas_laborales * 60,
            f"TiempoTratamiento_{e}_{d}"
        )


# Tiempos viajes
for p in poligonos:
    for d in dias:
        # Determinar jornada laboral
        day_of_week = (d - 1) % 7
        jornada = HORAS_LABOR_SABADO if day_of_week == 5 else HORAS_LABOR_DIA_LABORAL

        # Obtener tiempo de viaje ida y vuelta (en minutos)
        if p == "P18":
            tiempo_viaje = 0 + TIEMPO_CARGA_DESCARGA_CAMION_MIN # No se cuenta tiempo si es al mismo almacén
        else:
            tiempo_viaje = matriz_distancias.loc["P18", p] + TIEMPO_CARGA_DESCARGA_CAMION_MIN

        # Restricción: tiempo total de viajes al polígono p en día d no debe exceder jornada
        prob += pulp.lpSum(num_v[p, d] * tiempo_viaje) <= jornada, f"Tiempo_Viajes_{p}_{d}"

# Tiempos plantado

for e in especies:
    for p in poligonos:
        for d in dias:
            dia_semana = (d - 1) % 7  # Día de la semana (0 = lunes, ..., 6 = domingo)

            if dia_semana == 6:
                # No se planta en domingo
                prob += (
                    pulp.lpSum(pl[e, p, d, l] for l in dias if (e, p, d, l) in pl) == 0,
                    f"ProhibidoPlantado_Domingo_{e}_{p}_{d}"
                )
            else:
                horas_laborales = HORAS_LABOR_SABADO if dia_semana == 5 else HORAS_LABOR_DIA_LABORAL
                minutos_disponibles = horas_laborales * 60

                prob += (
                    pulp.lpSum(
                        pl[e, p, d, l] * 60/PLANTAS_POR_HORA_TRABAJO
                        for l in dias if (e, p, d, l) in pl
                    ) <= minutos_disponibles,
                    f"TiempoPlantado_{e}_{p}_{d}"
                )

# 8. Flujo de las plantas es menor a las que se transportadan a poligonos

for d in dias:
    for p in poligonos:
        for e in especies:
            plantadas = pulp.lpSum(pl[e, p, d, l] for l in dias if 3 <= (d - l) <= 7)
            transportadas = pulp.lpSum(c[e, p, d, v] for v in viajes_por_dia)

            prob += (
                plantadas <= transportadas,
                f"Flujo_Plantas_{e}_{p}_{d}"
            )

# Flujo plantas a poligonos ese dia menor o igual a las tratadas

for d in dias:
    for e in especies:
        transportadas_dia = pulp.lpSum(c[e, p, d, v] for p in poligonos for v in viajes_por_dia)
        tratadas_dia = pulp.lpSum(t[e, d])

        prob += (
            transportadas_dia <= tratadas_dia,
            f"Flujo_Transporte_Tratamiento_{e}_{d}"
        )

# Restriccion pedido a viveros

for v in viveros:
    for d in dias:
        prob += (
            pulp.lpSum(x[e, v, d] for e in especies if (e, v, d) in x) <= CAPACIDAD_CAMION_VIVERO * pi[v, d],
            f"CapacidadVivero_{v}_{d}"
        )



# Plantadas debe ser menor a las ordenadas hasta ese día
for e in especies:
    for d in dias:
        for l in dias:
            if 3 <= (d - l) <= 7:
                plantadas_limite = pulp.lpSum(pl[e, p, d, l] for p in poligonos)
                ordenadas_limite = pulp.lpSum(x[e, v, l] for v in viveros)

                prob += (
                    plantadas_limite <= ordenadas_limite,
                    f"Balance_Plantadas_Ordenadas_{e}_{d}_{l}"
                )


# Plantar al menos num plants por dias  # no funciona hace que se plante mas de lo que debe
# for p in poligonos:
#     for d in dias:
#         dia_semana = (d - 1) % 7  # 0 = lunes, ..., 6 = domingo

#         if dia_semana == 6:
#             minimo_plantas = 0  # Domingo
#         elif dia_semana == 5:
#             minimo_plantas = 40  # Sábado
#         else:
#             minimo_plantas = 90  # Lunes a viernes

#         prob += (
#             pulp.lpSum(
#                 pl[e, p, d, l]
#                 for e in especies
#                 for l in dias
#                 if (e, p, d, l) in pl and 3 <= (d - l) <= 7
#             ) >= minimo_plantas,
#             f"MinimoPlantado_{p}_{d}"
#         )

for d in dias:
    # Plantas sembradas en el día d (todas las especies, polígonos y llegadas válidas)
    plantadas_d = pulp.lpSum(
        pl[e, p, d, l]
        for e in especies
        for p in poligonos
        for l in dias
        if (e, p, d, l) in pl  # la variable existe
    )

    # Enlace Big-M
    prob += plantadas_d <= M_BIG * di[d],          f"DiaActivo_SUP_{d}"
    prob += plantadas_d >= 10*di[d],                  f"DiaActivo_INF_{d}"

MINIMO_PLANTAS_POR_PEDIDO = 10  # o el valor que decidas

for v in viveros:
    for d in dias:
        total_pedido = pulp.lpSum(
            x[e, v, d] for e in especies if (e, v, d) in x
        )

        # Restricción superior (máximo del camión)
        prob += total_pedido <= CAPACIDAD_CAMION_VIVERO * pedido_activo[v, d], f"CapacidadMax_{v}_{d}"

        # Restricción inferior (mínimo por pedido)
        prob += total_pedido >= MINIMO_PLANTAS_POR_PEDIDO * pedido_activo[v, d], f"CapacidadMin_{v}_{d}"

        # Enlace con la variable de conteo de pedidos (pi)
        prob += pi[v, d] >= pedido_activo[v, d], f"LinkPedidoActPi_{v}_{d}"

In [ ]:
prob.solve(solver)

Set parameter Threads to value 64
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 64 threads

         Reduce the value of the Threads parameter to improve performance


Non-default parameters:
MIPGap  0.3
Threads  64

Academic license 2677057 - for non-commercial use only - registered to a0___@itesm.mx
Optimize a model with 62200 rows, 471450 columns and 1136150 nonzeros
Model fingerprint: 0x1f6ed84d
Variable types: 0 continuous, 471450 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [5e-03, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 8e+06]
Presolve removed 48041 rows and 450077 columns
Presolve time: 0.38s
Presolved: 14159 rows, 21373 columns, 170181 nonzeros
Variable types: 0 continuous, 21373 integer (483 binary)
Performing another preso

1

In [ ]:
from pulp import LpStatus, value

# 1. Status of the solution
print(f"Status: {LpStatus[prob.status]}")

# 2. Objective value
print(f"Objective value (Costo total): {value(prob.objective):,.2f}")

Status: Optimal
Objective value (Costo total): 304,929.00


In [ ]:
costo

In [ ]:
def extraer_resultados_variable(variable_dict, nombre_columnas=None, nombre_valor="Valor"):
    """
    Convierte un diccionario de variables de PuLP en un DataFrame con valores mayores a cero.

    Parámetros:
        variable_dict (dict): Diccionario de variables de PuLP.
        nombre_columnas (list): Lista opcional de nombres para las columnas de los índices.
        nombre_valor (str): Nombre de la columna para el valor de la variable.

    Retorna:
        pd.DataFrame: DataFrame con los valores positivos de las variables.
    """
    resultados = []

    for indices, variable in variable_dict.items():
        valor = variable.varValue
        if valor is not None and valor > 0:
            if isinstance(indices, tuple):
                fila = dict(zip(nombre_columnas or [f"Index_{i}" for i in range(len(indices))], indices))
            else:
                fila = {nombre_columnas[0] if nombre_columnas else "Index_0": indices}
            fila[nombre_valor] = int(valor) if valor.is_integer() else valor
            resultados.append(fila)

    return pd.DataFrame(resultados)



In [ ]:
df_t = extraer_resultados_variable(t, nombre_columnas=["Especie", "Día"], nombre_valor="Cantidad_Tratada")
df_t

,Especie,Día,Cantidad_Tratada
0,E1,82,3
1,E1,83,3
2,E1,85,3
3,E1,86,7
4,E1,87,4
...,...,...,...
158,E10,93,9
159,E10,94,9
160,E10,95,9
161,E10,96,9


In [ ]:
df_x = extraer_resultados_variable(x, nombre_columnas=["Especie", "Vivero", "Día"], nombre_valor="Cantidad_Pedida")
df_x

,Especie,Vivero,Día,Cantidad_Pedida
0,E1,V4,82,3
1,E1,V4,86,7
2,E1,V4,89,32
3,E1,V4,92,16
4,E1,V4,93,59
5,E1,V4,97,1
6,E2,V4,82,35
7,E2,V4,86,70
8,E2,V4,89,202
9,E2,V4,92,296


In [ ]:
df_dia_act = extraer_resultados_variable(di, nombre_columnas=["Dia"], nombre_valor="Dia activo")
df_dia_act

,Dia,Dia activo
0,73,1
1,74,1
2,75,1
3,86,1
4,87,1
5,88,1
6,89,1
7,92,1
8,93,1
9,94,1


In [ ]:
df_x.Día.nunique()

10

In [ ]:
df_x.Cantidad_Pedida.sum()

np.int64(2645)

In [ ]:
df_almacen = extraer_resultados_variable(a, nombre_columnas=["Especie", "Día"], nombre_valor="Cantidad_Almacen")
df_almacen

,Especie,Día,Cantidad_Almacen
0,E1,82,3
1,E1,83,3
2,E1,84,3
3,E1,85,3
4,E1,86,7
...,...,...,...
206,E10,96,18
207,E10,97,18
208,E10,98,18
209,E10,99,9


In [ ]:
df_pl = extraer_resultados_variable(pl, nombre_columnas=["Especie", "Poligono","Dia","Dia de llegada"], nombre_valor="Cantidad_Plantada")
df_pl

,Especie,Poligono,Dia,Dia de llegada,Cantidad_Plantada
0,E1,P17,86,82,2
1,E1,P17,89,82,2
2,E1,P17,92,86,1
3,E1,P17,93,89,28
4,E1,P17,94,89,6
...,...,...,...,...,...
177,E10,P18,93,89,7
178,E10,P19,92,89,9
179,E10,P19,93,89,2
180,E10,P19,94,89,9


In [ ]:
df_pl.Cantidad_Plantada.sum()

np.int64(2529)

In [ ]:
df_t['Día'].nunique()

27

In [ ]:
# from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=df_pl)

In [ ]:
df_pl.Cantidad_Plantada.sum()

np.int64(2529)

In [ ]:
df_pi = extraer_resultados_variable(pi, nombre_columnas=["Vivero","Dia"], nombre_valor="Pedidos")
df_pi

,Vivero,Dia,Pedidos
0,V1,69,1
1,V1,83,1
2,V1,89,1
3,V1,95,1
4,V2,83,1
5,V2,86,1
6,V3,83,1
7,V3,87,1
8,V4,82,1
9,V4,86,1


In [ ]:
df_t.to_csv(f"df_t.csv", index=False)
df_x.to_csv(f"df_x.csv", index=False)
df_dia_act.to_csv(f"df_dia_act.csv", index=False)
df_almacen.to_csv(f"df_almacen.csv", index=False)
df_pl.to_csv(f"df_pl.csv", index=False)
df_pi.to_csv(f"df_pi.csv", index=False)

# Enfoque por meses.

In [ ]:
def construir_y_resolver_modelo(dias_mes, min_tipo, min_poligono):
  # --- Optimization Model Structure (PuLP) ---
  # Create the problem
  mes_prob = pulp.LpProblem("Reforestation_Optimization", pulp.LpMinimize)

  # --- Decision Variables ---

  #   1. Nùmero de plantas e del vivero v que se pidieron el dia d

  x = pulp.LpVariable.dicts(
      "NumPlantas",
      ((e, v, d) for e in especies for v in viveros for d in dias),
      lowBound=0,
      cat='Integer'
  )

  #   2. Número de plantas e en el almacen en el dia d

  a = pulp.LpVariable.dicts(
      "PlantasAlmacen",
      ((e, d) for e in especies for d in dias),
      lowBound=0,
      cat='Integer'
  )

  #   3. Numero de plantas e que se tratan en dia d

  t = pulp.LpVariable.dicts(
      "PlantasTratadas",
      ((e, d) for e in especies for d in dias),
      lowBound=0,
      cat='Integer'
  )

  #   4. Numero de plantas e que se sembraron en el poligono p en el dia d que llegaron el dia l

  pl = pulp.LpVariable.dicts(
      "PlantasSembradas",
      ((e, p, d, l) for e in especies for p in poligonos for d in dias for l in dias if 3 <= (d - l) <= 7),
      lowBound=0,
      cat='Integer'
  )

  #   5. Numero de viajes al poligono p en el dia d

  num_v = pulp.LpVariable.dicts(
      "ViajesPoligono",
      ((p, d) for p in poligonos for d in dias),
      lowBound=0,
      cat='Integer'
  )

  #   6. Variable binaria que establece si ese dia se va al poligono p

  vi = pulp.LpVariable.dicts(
      "VisitaPoligono",
      ((p, d) for p in poligonos for d in dias),
      cat='Binary'
  )

  #   7. Numero de plantas e que se llevaron al poligono p en el dia d en el viaje v

  c = pulp.LpVariable.dicts(
      "PlantasTransportadasViaje",
      ((e, p, d, viaje) for e in especies for p in poligonos for d in dias for viaje in viajes_por_dia),
      lowBound=0,
      cat='Integer'
  )

  #   8. Numero de veces que se pidio en el dia d al vivero v
  pi = pulp.LpVariable.dicts(
      "PedidosVivero",
      ((v, d) for v in viveros for d in dias),
      lowBound=0,
      cat='Integer'
  )

  di = pulp.LpVariable.dicts("DiaActivo", dias, cat='Binary')


  pedido_activo = pulp.LpVariable.dicts(
      "PedidoActivo",
      ((v, d) for v in viveros for d in dias),
      cat='Binary'
  )

  # --- Objective Function ---

  mes_prob += (
      # Costo por pedir plantas al vivero (pi * costo fijo por pedido)
      pulp.lpSum(pi[(v, d)] * COSTO_CAMION_VIVERO for v in viveros for d in dias)
      +
      # Costo por plantas pedidas (sumatoria x * costo planta)
      pulp.lpSum(x[(e, v, d)] * costos_viveros[v][e] for e in especies for v in viveros for d in dias)
      +
      # Costo por plantar plantas (cuádruple sumatoria pl * costo de plantar)
      pulp.lpSum(pl[(e, p, d, l)] * COSTO_PLANTACION_POR_PLANTA for e in especies for p in poligonos for d in dias for l in dias if (e, p, d, l) in pl)
      +
      # Costo gasolina: (distancia / rendimiento) * precio * número de viajes v(p,d)
      pulp.lpSum(
          (distancia_almacen_poligono_km[p] / RENDIMIENTO_LITRO_CAMIONETA) * PRECIO_GASOLINA * num_v[(p, d)]
          for p in poligonos for d in dias
      )
      + COSTO_DIA_ACTIVO * pulp.lpSum(di[d] for d in dias)
  ), "CostoTotal"

  #### restricciones
# 1. Pide al menos lo minimo para sembrar las hectareas. especies

  for e in especies:
      mes_prob += pulp.lpSum(pl[(e, p, d, l)]
                        for p in poligonos
                        for d in dias
                        for l in dias
                        if (e, p, d, l) in pl and 3 <= (d - l) <= 7) >= min_tipo[e], f"Minimo_Plantas_{e}"


  # 2. Pide al menos lo minimo para sembrar las hectareas. poligonos

  # for p in poligonos:
  #     mes_prob += pulp.lpSum(
  #         pl[(e, p, d, l)]
  #         for e in especies
  #         for d in dias
  #         for l in dias
  #         if (e, p, d, l) in pl and 3 <= (d - l) <= 7
  #     ) >= min_poligono[p], f"Minimo_Plantas_{p}"


  # 2. Lo que plantas no debe ser mayor a lo que tratas

  for e in especies:
      for d in dias:
          mes_prob += t[(e, d)] >= pulp.lpSum(
              pl[(e, p, d, l)]
              for p in poligonos
              for l in dias
              if (e, p, d, l) in pl and 3 <= (d - l) <= 7
          ), f"Tratamiento_Suficiente_{e}_{d}"

  #3. No puedes tratar mas de lo que tienes en almacen

  for e in especies:
      for d in dias:
          mes_prob += a[e, d] >= t[e, d], f"Tratado_no_excede_almacen_{e}_{d}"

  #4. Flujo del almacen

  for e in especies:
      for d in dias:
          if d == 1:
              mes_prob += a[e, d] == 0, f"Inicial_almacen_{e}" # Dia cero del almacen , se inicializa en 0
          else:
              entradas = pulp.lpSum(
                  x[e, v, d - TIEMPO_ANTICIPACION_PEDIDO_DIAS]
                  for v in viveros
                  if (e, v, d - TIEMPO_ANTICIPACION_PEDIDO_DIAS) in x
              )
              salidas = pulp.lpSum(
                  pl[e, p, d, l]
                  for p in poligonos
                  for l in dias
                  if (e, p, d, l) in pl
              )
              mes_prob += a[e, d] == a[e, d - 1] + entradas - salidas, f"Flujo_almacen_{e}_{d}"

  #5. Volumen

  # Usar esta cuando se tenga un volumen diferente por cada tipo de planta

  for d in dias:
      mes_prob += (
          pulp.lpSum(a[e, d] * VOLUMEN_PLANTA_CM3[e]) <= CAPACIDAD_ALMACEN_CM3,
          f"Restriccion_Volumen_Almacen_dia_{d}"
      )

  for d in dias:
      for p in poligonos:
          for v in viajes_por_dia:
              mes_prob += (pulp.lpSum(c[e,p,d,v] * VOLUMEN_PLANTA_CM3[e]) <=  VOLUMEN_CAMION_CM3   ,f"Restriccion_Volumen_Camioneta_{p}_{d}_{v}")

  for d in dias:
      mes_prob += (
          pulp.lpSum(t[e, d] * AREA_BASE_PLANTA_CM2) <= AREA_INSTALACION_TRATAMIENTO_CM2 ,f"Restriccion_Volumen_Tratado_{e}_{d}_")


  # No pueden aparecer plantas mágicas

  for e in especies:
      total_plantadas = pulp.lpSum(pl[e, p, d, l] for p in poligonos for d in dias for l in dias if 3 <= (d - l) <= 7)
      total_pedidas = pulp.lpSum(x[e, v, d] for v in viveros for d in dias)

      mes_prob += (
          total_plantadas <= total_pedidas,
          f"Conservacion_Plantas_{e}"
      )

  # Tiempos tratados

  for d in dias:
      # Determinar si el día es sábado (5) o un día entre semana (0–4)
      dia_semana = (d - 1) % 7  # Asumiendo día 1 es lunes
      horas_laborales = HORAS_LABOR_SABADO if dia_semana == 5 else HORAS_LABOR_DIA_LABORAL
      mes_prob += (
              t[e, d] * PROMEDIO_TRATADO_PLANTAS <= horas_laborales * 60,
              f"TiempoTratamiento_{e}_{d}"
          )

  # # Tiempos viajes

  for p in poligonos:
      if p == "P18":
          continue  # No se hacen viajes del almacén al almacén

      for d in dias:
          # Determinar jornada laboral
          day_of_week = (d - 1) % 7
          jornada = HORAS_LABOR_SABADO if day_of_week == 5 else HORAS_LABOR_DIA_LABORAL

          # Obtener tiempo de viaje ida y vuelta (en minutos)
          tiempo_viaje = matriz_distancias.loc["P18", p] + TIEMPO_CARGA_DESCARGA_CAMION_MIN

          # Restricción: tiempo total de viajes al polígono p en día d no debe exceder jornada
          mes_prob += pulp.lpSum(num_v[p, d] * tiempo_viaje) <= jornada, f"Tiempo_Viajes_{p}_{d}"

  #Tiempos plantado

  for e in especies:
      for p in poligonos:
          for d in dias:
              dia_semana = (d - 1) % 7  # Día de la semana (0 = lunes, ..., 6 = domingo)

              if dia_semana == 6:
                  # No se planta en domingo
                  mes_prob += (
                      pulp.lpSum(pl[e, p, d, l] for l in dias if (e, p, d, l) in pl) == 0,
                      f"ProhibidoPlantado_Domingo_{e}_{p}_{d}"
                  )
              else:
                  horas_laborales = HORAS_LABOR_SABADO if dia_semana == 5 else HORAS_LABOR_DIA_LABORAL
                  minutos_disponibles = horas_laborales * 60

                  mes_prob += (
                      pulp.lpSum(
                          pl[e, p, d, l] * 60/PLANTAS_POR_HORA_TRABAJO
                          for l in dias if (e, p, d, l) in pl
                      ) <= minutos_disponibles,
                      f"TiempoPlantado_{e}_{p}_{d}"
                )

  # 8. Flujo de las plantas es menor a las que se transportadan a poligonos

  for d in dias:
      for p in poligonos:
          for e in especies:
              plantadas = pulp.lpSum(pl[e, p, d, l] for l in dias if 3 <= (d - l) <= 7)
              transportadas = pulp.lpSum(c[e, p, d, v] for v in viajes_por_dia)

              mes_prob += (
                  plantadas <= transportadas,
                  f"Flujo_Plantas_{e}_{p}_{d}"
              )

  # Flujo plantas a poligonos ese dia menor o igual a las tratadas

  for d in dias:
      for e in especies:
          transportadas_dia = pulp.lpSum(c[e, p, d, v] for p in poligonos for v in viajes_por_dia)
          tratadas_dia = pulp.lpSum(t[e, d])

          mes_prob += (
              transportadas_dia <= tratadas_dia,
              f"Flujo_Transporte_Tratamiento_{e}_{d}"
          )

  # Restriccion pedido a viveros

  for v in viveros:
      for d in dias:
          mes_prob += (
              pulp.lpSum(x[e, v, d] for e in especies if (e, v, d) in x) <= CAPACIDAD_CAMION_VIVERO * pi[v, d],
              f"CapacidadVivero_{v}_{d}"
          )



  # Plantadas debe ser menor a las ordenadas hasta ese día
  for e in especies:
      for d in dias:
          for l in dias:
              if 3 <= (d - l) <= 7:
                  plantadas_limite = pulp.lpSum(pl[e, p, d, l] for p in poligonos)
                  ordenadas_limite = pulp.lpSum(x[e, v, l] for v in viveros)

                  mes_prob += (
                      plantadas_limite <= ordenadas_limite,
                      f"Balance_Plantadas_Ordenadas_{e}_{d}_{l}"
                  )


  # Plantar al menos num plants por dias  # no funciona hace que se plante mas de lo que debe

  # for p in poligonos:
  #     for d in dias:
  #         dia_semana = (d - 1) % 7  # 0 = lunes, ..., 6 = domingo

  #         if dia_semana == 6:
  #             minimo_plantas = 0  # Domingo
  #         elif dia_semana == 5:
  #             minimo_plantas = 20  # Sábado
  #         else:
  #             minimo_plantas = 50  # Lunes a viernes

  #         mes_prob += (
  #             pulp.lpSum(
  #                 pl[e, p, d, l]
  #                 for e in especies
  #                 for l in dias
  #                 if (e, p, d, l) in pl and 3 <= (d - l) <= 7
  #             ) >= minimo_plantas,
  #             f"MinimoPlantado_{p}_{d}"
  #         )

  for d in dias:
      # Plantas sembradas en el día d (todas las especies, polígonos y llegadas válidas)
      plantadas_d = pulp.lpSum(
          pl[e, p, d, l]
          for e in especies
          for p in poligonos
          for l in dias
          if (e, p, d, l) in pl  # la variable existe
      )

      # Enlace Big-M
      mes_prob += plantadas_d <= M_BIG * di[d],          f"DiaActivo_SUP_{d}"
      mes_prob += plantadas_d >= 1*di[d],                  f"DiaActivo_INF_{d}"

  MINIMO_PLANTAS_POR_PEDIDO = 10  # o el valor que decidas

  for v in viveros:
      for d in dias:
          total_pedido = pulp.lpSum(
              x[e, v, d] for e in especies if (e, v, d) in x
          )

          # Restricción superior (máximo del camión)
          mes_prob += total_pedido <= CAPACIDAD_CAMION_VIVERO * pedido_activo[v, d], f"CapacidadMax_{v}_{d}"

          # Restricción inferior (mínimo por pedido)
          mes_prob += total_pedido >= MINIMO_PLANTAS_POR_PEDIDO * pedido_activo[v, d], f"CapacidadMin_{v}_{d}"

          # Enlace con la variable de conteo de pedidos (pi)
          mes_prob += pi[v, d] >= pedido_activo[v, d], f"LinkPedidoActPi_{v}_{d}"

  status = mes_prob.solve(solver)
  valor_objetivo = pulp.value(mes_prob.objective)

  return {
    "prob": mes_prob,
    "status": status,
    "valor_objetivo": valor_objetivo,
    "x": x,
    "pl": pl,
    "a": a,
    "t": t,
    "pi": pi,
    "c": c,
    "dias": dias  # útil para exportar resultados
}

In [ ]:
dias_por_mes = {}
num_meses = len(dias) // 30
for i in range(num_meses):
    nombre_mes = f"Mes_{i+1}"
    dias_por_mes[nombre_mes] = dias[i*30:(i+1)*30]
dias_por_mes

In [ ]:
from copy import deepcopy
import pandas as pd
import os

# Carpeta donde guardar resultados
os.makedirs("resultados_mensuales", exist_ok=True)

min_tipo_restante = deepcopy(MIN_PLANTAS_POR_TIPO)
min_poligono_restante = deepcopy(MIN_PLANTAS_POR_POLIGONO)


# Crear copia de los requerimientos originales
NUM_MESES = 7
# Divide el requerimiento total por mes
min_tipo_por_mes = {
    f"Mes_{i+1}": {e: int(MIN_PLANTAS_POR_TIPO[e] / NUM_MESES) for e in MIN_PLANTAS_POR_TIPO}
    for i in range(NUM_MESES)
}

min_poligono_por_mes = {
    f"Mes_{i+1}": {p: int(MIN_PLANTAS_POR_POLIGONO[p] / NUM_MESES) for p in MIN_PLANTAS_POR_POLIGONO}
    for i in range(NUM_MESES)
}

# Lista para almacenar resultados de cada mes
resultados_mensuales = []



In [ ]:

for mes, dias_mes in dias_por_mes.items():
    print(f"\n🔄 Ejecutando modelo para {mes}...")
    dias_mes=range(1,100)
    min_tipo_actual = min_tipo_por_mes[mes]
    print(min_tipo_actual)
    min_poligono_actual = min_poligono_por_mes[mes]

    resultado = construir_y_resolver_modelo(dias_mes, min_tipo_actual, min_poligono_actual)

    # Ejecuta la función usando solo los días del mes y requerimientos restantes
    # Extrae valor objetivo
    print(f"✅ {mes} - Status: {pulp.LpStatus[resultado['status']]}, Objetivo: {resultado['valor_objetivo']:.2f}")

    # Exporta el modelo LP
    resultado["prob"].writeLP(f"resultados_mensuales/{mes}_modelo.lp")

    # Guarda valor objetivo
    with open(f"resultados_mensuales/{mes}_objetivo.txt", "w") as f:
        f.write(f"{pulp.LpStatus[resultado['status']]} - Objetivo: {resultado['valor_objetivo']:.2f}")

    # Extraer y guardar resultados en CSVs
    for nombre_var in ['x', 'pl', 'a', 't', 'pi', 'c']:
        var = resultado[nombre_var]
        registros = [
            list(k) + [v.varValue]
            for k, v in var.items()
            if v.varValue is not None and v.varValue > 0
        ]
        if registros:
            df = pd.DataFrame(registros)
            df.to_csv(f"resultados_mensuales/{mes}_{nombre_var}.csv", index=False)
            resultado[nombre_var + '_df'] = df

    # Actualiza requerimientos restantes
    pl = resultado['pl']
    for (e, p, d, l), var in pl.items():
        if var.varValue and var.varValue > 0:
            min_tipo_restante[e] = max(0, min_tipo_restante[e] - var.varValue)
            min_poligono_restante[p] = max(0, min_poligono_restante[p] - var.varValue)

    # Guarda para consolidar luego
    resultados_mensuales.append({
        "mes": mes,
        "valor_objetivo": resultado["valor_objetivo"],
        "dataframes": {k: resultado[k + '_df'] for k in ['x', 'pl', 'a', 't', 'pi', 'c'] if k + '_df' in resultado}
    })

# (Opcional) Unir todos los resultados en archivos unificados por variable
for var in ['x', 'pl', 'a', 't', 'pi', 'c']:
    dfs = []
    for r in resultados_mensuales:
        if var in r["dataframes"]:
            df = r["dataframes"][var]
            df["mes"] = r["mes"]
            dfs.append(df)
    if dfs:
        df_total = pd.concat(dfs, ignore_index=True)
        df_total.to_csv(f"resultados_mensuales/UNIFICADO_{var}.csv", index=False)